In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 5.5 MB/s eta 0:00:00


In [3]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00


In [4]:
!pip install scikeras

In [5]:
# Import libraries
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/7_GDC (Grouped Dipeptide Composition)/GDC_validation_negative.csv")

# Prepare features and labels for training
X_train = pd.concat([main_p, main_n])
y_train = pd.concat([pd.Series(1, index=main_p.index), pd.Series(0, index=main_n.index)])


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [6]:
# Define a cross-validation accuracy function
def cross_val_accuracy(model, X_train, y_train):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    return accuracies.mean()


In [7]:
#Support Vector Machine (SVM)
def objective_svm(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)

    model = SVC(kernel=kernel, C=C, gamma=gamma)
    return cross_val_accuracy(model, X_train, y_train)

study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=50)
print("SVM Best Parameters:", study_svm.best_params)
print("SVM Best Accuracy:", study_svm.best_value)


[I 2024-12-07 10:04:47,179] A new study created in memory with name: no-name-da9766c2-d44a-4d1b-9d0b-310b5db9379b
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
<ipython-input-7-cd4fae0f87b1>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)
[I 2024-12-07 10:04:47,709] Trial 0 finished with value: 0.5250073997336095 and parameters: {'kernel': 'rbf', 'C': 0.00028144398081643413, 'gamma': 1.9304786167728936e-05}. Best is trial 0 with value: 0.5250073997336095.
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been depreca

SVM Best Parameters: {'kernel': 'linear', 'C': 1722.7621479826594, 'gamma': 0.00024049707532865157}
SVM Best Accuracy: 0.731045582359035


In [8]:
#Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)
print("Random Forest Best Parameters:", study_rf.best_params)
print("Random Forest Best Accuracy:", study_rf.best_value)


[I 2024-12-07 10:08:14,912] A new study created in memory with name: no-name-dedb2d16-f27c-4ba5-aca8-0da22d16a6ec
[I 2024-12-07 10:08:17,807] Trial 0 finished with value: 0.8255919786887673 and parameters: {'n_estimators': 231, 'max_depth': 29, 'min_samples_split': 15, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8255919786887673.
[I 2024-12-07 10:08:19,134] Trial 1 finished with value: 0.8333357999112032 and parameters: {'n_estimators': 107, 'max_depth': 12, 'min_samples_split': 8, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.8333357999112032.
[I 2024-12-07 10:08:20,147] Trial 2 finished with value: 0.8161388190025161 and parameters: {'n_estimators': 86, 'max_depth': 27, 'min_samples_split': 5, 'min_samples_leaf': 9}. Best is trial 1 with value: 0.8333357999112032.
[I 2024-12-07 10:08:22,031] Trial 3 finished with value: 0.7619986680479502 and parameters: {'n_estimators': 208, 'max_depth': 3, 'min_samples_split': 16, 'min_samples_leaf': 6}. Best is trial 1 with value

Random Forest Best Parameters: {'n_estimators': 294, 'max_depth': 19, 'min_samples_split': 3, 'min_samples_leaf': 1}
Random Forest Best Accuracy: 0.8582581027083025


In [9]:
#Decision Tree
def objective_dt(trial):
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective_dt, n_trials=50)
print("Decision Tree Best Parameters:", study_dt.best_params)
print("Decision Tree Best Accuracy:", study_dt.best_value)


[I 2024-12-07 10:11:03,963] A new study created in memory with name: no-name-0ea99bd1-3e0e-4912-b8fb-c6d0e15b905f
[I 2024-12-07 10:11:04,017] Trial 0 finished with value: 0.7276454047654285 and parameters: {'max_depth': 4, 'min_samples_split': 19, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7276454047654285.
[I 2024-12-07 10:11:04,061] Trial 1 finished with value: 0.7276454047654285 and parameters: {'max_depth': 4, 'min_samples_split': 17, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7276454047654285.
[I 2024-12-07 10:11:04,114] Trial 2 finished with value: 0.8256104780227911 and parameters: {'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.8256104780227911.
[I 2024-12-07 10:11:04,180] Trial 3 finished with value: 0.8127275418084949 and parameters: {'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.8256104780227911.
[I 2024-12-07 10:11:04,238] Trial 4 finished with value: 0.8110

Decision Tree Best Parameters: {'max_depth': 12, 'min_samples_split': 8, 'min_samples_leaf': 2}
Decision Tree Best Accuracy: 0.828196684919343


In [10]:
#k-Nearest Neighbors (k-NN)
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm
    )
    return cross_val_accuracy(model, X_train, y_train)

study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=50)
print("k-NN Best Parameters:", study_knn.best_params)
print("k-NN Best Accuracy:", study_knn.best_value)


[I 2024-12-07 10:11:08,804] A new study created in memory with name: no-name-5fa05caf-e957-425a-8acf-afa487d2bc79
[I 2024-12-07 10:11:08,901] Trial 0 finished with value: 0.8281929850525381 and parameters: {'n_neighbors': 12, 'weights': 'distance', 'algorithm': 'auto'}. Best is trial 0 with value: 0.8281929850525381.
[I 2024-12-07 10:11:08,984] Trial 1 finished with value: 0.8316042622465591 and parameters: {'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'auto'}. Best is trial 1 with value: 0.8316042622465591.
[I 2024-12-07 10:11:09,152] Trial 2 finished with value: 0.8384897143702826 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'brute'}. Best is trial 2 with value: 0.8384897143702826.
[I 2024-12-07 10:11:09,239] Trial 3 finished with value: 0.8247558087908835 and parameters: {'n_neighbors': 12, 'weights': 'distance', 'algorithm': 'brute'}. Best is trial 2 with value: 0.8384897143702826.
[I 2024-12-07 10:11:09,310] Trial 4 finished with value: 0.8307680

k-NN Best Parameters: {'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'ball_tree'}
k-NN Best Accuracy: 0.8410685215332248


In [11]:
#Logistic Regression

def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    penalty = trial.suggest_categorical('penalty', ['l2'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])

    model = LogisticRegression(C=C, penalty=penalty, solver=solver)
    return cross_val_accuracy(model, X_train, y_train)

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=50)
print("Logistic Regression Best Parameters:", study_lr.best_params)
print("Logistic Regression Best Accuracy:", study_lr.best_value)


[I 2024-12-07 10:11:13,793] A new study created in memory with name: no-name-d969d6fe-4736-4baa-8cdb-643a8dc991b3
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-07 10:11:13,869] Trial 0 finished with value: 0.6941209116471808 and parameters: {'C': 4.405762927484848, 'penalty': 'l2', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.6941209116471808.
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-07 10:11:13,999] Trial 1 finished with value: 0.7138708006511765 and parameters: {'C': 53503.377710

Logistic Regression Best Parameters: {'C': 39.23812020480505, 'penalty': 'l2', 'solver': 'lbfgs'}
Logistic Regression Best Accuracy: 0.721607222140003


In [12]:
#Naive Bayes
def objective_nb(trial):
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)

    model = GaussianNB(var_smoothing=var_smoothing)
    return cross_val_accuracy(model, X_train, y_train)

study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)
print("Naive Bayes Best Parameters:", study_nb.best_params)
print("Naive Bayes Best Accuracy:", study_nb.best_value)


[I 2024-12-07 10:11:18,003] A new study created in memory with name: no-name-710be5ec-4bd2-4bb4-8bce-b8a66ef8b12a
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-07 10:11:18,049] Trial 0 finished with value: 0.6795212372354595 and parameters: {'var_smoothing': 7.902169717293363e-08}. Best is trial 0 with value: 0.6795212372354595.
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-07 10:11:18,085] Trial 1 finished with value: 0.6795212372354595 and 

Naive Bayes Best Parameters: {'var_smoothing': 0.02274127153699435}
Naive Bayes Best Accuracy: 0.6889743969217108


In [13]:
#Gradient Boosting
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_gb = optuna.create_study(direction='maximize')
study_gb.optimize(objective_gb, n_trials=50)
print("Gradient Boosting Best Parameters:", study_gb.best_params)
print("Gradient Boosting Best Accuracy:", study_gb.best_value)


[I 2024-12-07 10:11:20,561] A new study created in memory with name: no-name-99133394-a94c-4113-93ff-ea3bab182f45
<ipython-input-13-0f44ec352f28>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-13-0f44ec352f28>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-07 10:11:32,860] Trial 0 finished with value: 0.8428000591978687 and parameters: {'n_estimators': 102, 'learning_rate': 0.0001760673511840182, 'max_depth': 15, 'subsample': 0.9455747139512779}. Best is trial 0 with value: 0.8428000591978687.
<ipython-input-13-0f44ec352f28>:4: Futur

Gradient Boosting Best Parameters: {'n_estimators': 142, 'learning_rate': 0.09830358294659793, 'max_depth': 17, 'subsample': 0.6397598173179956}
Gradient Boosting Best Accuracy: 0.8651398549652214


In [14]:
#XGBoost

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("XGBoost Best Parameters:", study_xgb.best_params)
print("XGBoost Best Accuracy:", study_xgb.best_value)


[I 2024-12-07 10:20:39,268] A new study created in memory with name: no-name-8ec983a6-d93d-4e1a-b194-d53d4b015910
<ipython-input-14-ad894fc3f237>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-14-ad894fc3f237>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-07 10:20:39,979] Trial 0 finished with value: 0.8247225099896405 and parameters: {'n_estimators': 60, 'max_depth': 15, 'learning_rate': 0.014963066420465068, 'subsample': 0.5749097406401387}. Best is trial 0 with value: 0.8247225099896405.
<ipython-input-14-ad894fc3f237>:6: FutureW

XGBoost Best Parameters: {'n_estimators': 153, 'max_depth': 19, 'learning_rate': 0.09532019680018873, 'subsample': 0.5646609469718473}
XGBoost Best Accuracy: 0.8634083173005772


In [15]:
# LightGBM
def objective_lgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 50)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        num_leaves=num_leaves
    )
    return cross_val_accuracy(model, X_train, y_train)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
print("LightGBM Best Parameters:", study_lgb.best_params)
print("LightGBM Best Accuracy:", study_lgb.best_value)


[I 2024-12-07 10:21:57,891] A new study created in memory with name: no-name-fa8fb7c7-b06a-49f1-91eb-f8fdfba21a46
<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000827 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 552
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000138 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 546
[LightGBM] [Info] Number of data points in the tr

[I 2024-12-07 10:21:58,709] Trial 0 finished with value: 0.7938101228355778 and parameters: {'n_estimators': 172, 'max_depth': 8, 'learning_rate': 0.00013685225478608047, 'num_leaves': 33}. Best is trial 0 with value: 0.7938101228355778.


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000171 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 552
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


...


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000180 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 552
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-12-07 10:22:24,717] Trial 49 finished with value: 0.8539625573479356 and parameters: {'n_estimators': 182, 'max_depth': 6, 'learning_rate': 0.07387560116521191, 'num_leaves': 23}. Best is trial 49 with value: 0.8539625573479356.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
LightGBM Best Parameters: {'n_estimators': 182, 'max_depth': 6, 'learning_rate': 0.07387560116521191

In [16]:
#AdaBoost
def objective_ab(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    model = AdaBoostClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate
    )
    return cross_val_accuracy(model, X_train, y_train)

study_ab = optuna.create_study(direction='maximize')
study_ab.optimize(objective_ab, n_trials=50)
print("AdaBoost Best Parameters:", study_ab.best_params)
print("AdaBoost Best Accuracy:", study_ab.best_value)


[I 2024-12-07 10:22:24,750] A new study created in memory with name: no-name-3c47bbed-eb2e-4c01-80c4-3e64e187ab7f
<ipython-input-16-95971a9e6870>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: Futu

AdaBoost Best Parameters: {'n_estimators': 164, 'learning_rate': 0.0881623144237924}
AdaBoost Best Accuracy: 0.7602930294509397


In [17]:
#Neural Network

import optuna
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Load your dataset
# Replace this with your actual data loading process
X = np.random.rand(1000, 20)  # Dummy features
y = np.random.randint(0, 2, 1000)  # Dummy binary labels

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_nn(trial):
    # Define hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # Build the neural network
    model = Sequential()
    model.add(Dense(hidden_units, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dropout(dropout_rate))

    for _ in range(num_layers - 1):
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model (without validation data)
    history = model.fit(X, y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)

    # Evaluate the model
    train_accuracy = max(history.history['accuracy'])
    return train_accuracy

# Run Optuna study
study_nn = optuna.create_study(direction='maximize')
study_nn.optimize(objective_nn, n_trials=50)

# Print the best parameters and accuracy
print("Neural Network Best Parameters:", study_nn.best_params)
print("Neural Network Best Accuracy:", study_nn.best_value)

[I 2024-12-07 10:24:16,770] A new study created in memory with name: no-name-81b874a0-c061-42d7-b659-329e1def9b3f
<ipython-input-17-565ccca4d2fe>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
<ipython-input-17-565ccca4d2fe>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activit

Neural Network Best Parameters: {'num_layers': 2, 'hidden_units': 448, 'dropout_rate': 0.1309869450905613, 'learning_rate': 0.001836622897049772, 'batch_size': 16, 'epochs': 45}
Neural Network Best Accuracy: 1.0


In [18]:
#Multi-Layer Perceptron (MLP)

import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Load your dataset (replace this with your actual dataset)
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Normalize the dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_mlp(trial):
    # Define the hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 50, 200)

    # Define the MLP model
    model = MLPClassifier(
        hidden_layer_sizes=(hidden_units,) * num_layers,
        activation=activation,
        learning_rate_init=learning_rate,
        max_iter=epochs,
        batch_size=batch_size,
        solver='adam',
        random_state=42
    )

    # Evaluate using cross-validation
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Run Optuna study
study_mlp = optuna.create_study(direction='maximize')
study_mlp.optimize(objective_mlp, n_trials=50)

# Print results
print("MLP Best Parameters:", study_mlp.best_params)
print("MLP Best Accuracy:", study_mlp.best_value)

[I 2024-12-07 10:36:13,871] A new study created in memory with name: no-name-41c0892a-7b81-48a7-977c-701d6fc8927b
<ipython-input-18-5ee0db595401>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (155) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (155) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: S

MLP Best Parameters: {'num_layers': 2, 'hidden_units': 448, 'activation': 'logistic', 'learning_rate': 0.0018913677237848738, 'batch_size': 16, 'epochs': 69}
MLP Best Accuracy: 0.8770000000000001


probabilites dataset

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Combine data and create labels
X_train = pd.concat([main_p, main_n], axis=0)
y_train = np.array([1] * len(main_p) + [0] * len(main_n))

X_validation = pd.concat([validation_p, validation_n], axis=0)
y_validation = np.array([1] * len(validation_p) + [0] * len(validation_n))

# Define models
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    #"CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42)
}

# Combined probabilities dataset
combined_probabilities = pd.DataFrame(index=X_validation.index)

# Train models and get probabilities
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    probabilities = model.predict_proba(X_validation)[:, 1]
    combined_probabilities[model_name] = probabilities

# Neural Network
def build_nn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train Neural Network
input_dim = X_train.shape[1]
nn_model = build_nn(input_dim)
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_validation, y_validation), verbose=1)

# Predict probabilities for Neural Network
nn_probabilities = nn_model.predict(X_validation).flatten()
combined_probabilities['NeuralNetwork'] = nn_probabilities

# Add ground truth
combined_probabilities['True_Label'] = y_validation

# Save the combined probabilities dataset
output_path = "GDC_optuna.csv"
combined_probabilities.to_csv(output_path, index=False)
print(f"Combined probabilities dataset with Neural Network saved to {output_path}")


Training RandomForest...
Training GradientBoosting...
Training LogisticRegression...
Training DecisionTree...
Training NaiveBayes...
Training k-NN...
Training SVM...
Training XGBoost...
Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000235 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 598
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training AdaBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:34:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training MLP...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5721 - loss: 0.6797 - val_accuracy: 0.6833 - val_loss: 0.6271
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6808 - loss: 0.6177 - val_accuracy: 0.6967 - val_loss: 0.5715
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7076 - loss: 0.5723 - val_accuracy: 0.6900 - val_loss: 0.5574
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7017 - loss: 0.5621 - val_accuracy: 0.7067 - val_loss: 0.5508
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6664 - loss: 0.5638 - val_accuracy: 0.7000 - val_loss: 0.5485
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7183 - loss: 0.5491 - val_accuracy: 0.6967 - val_loss: 0.5430
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6791 - loss: 0.5582 - val_accuracy: 0.7133 - val_loss: 0.5397
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6855 - loss: 0.5464 - val_accuracy: 0.7133 - val_loss: 0.5330
Ep